In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

True

#Load raw PDF(s)

In [ ]:
DATA_PATH="./data/"
def load_pdf_files(data):
    loader = DirectoryLoader(data,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)
    
    documents=loader.load()
    return documents

documents=load_pdf_files(data=DATA_PATH)
print("Length of PDF pages: ", len(documents))

Length of PDF pages:  59


#Create Chunks

In [ ]:
def create_chunks(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=100,
                                                 chunk_overlap=25)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks=create_chunks(extracted_data=documents)
print("Length of Text Chunks: ", len(text_chunks))

Length of Text Chunks:  1599


#Create Vector Embeddings

In [ ]:
def get_embedding_model():
    embedding_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding_model

embedding_model=get_embedding_model()

# Step 4: Store embeddings in FAISS
DB_FAISS_PATH="vectorstore/db_faiss"
db=FAISS.from_documents(text_chunks, embedding_model)
db.save_local(DB_FAISS_PATH)

/Users/harshitchaudhari/Downloads/my_Playground_/sample/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings

HF_TOKEN = os.environ.get("HF_TOKEN")
HUGGINGFACE_REPO_ID = "mistralai/Mistral-7B-Instruct-v0.3"

def load_llm(huggingface_repo_id):
    if not HF_TOKEN:
        raise ValueError("HF_TOKEN environment variable is not set!")
    
    llm = HuggingFaceEndpoint(
        repo_id=huggingface_repo_id,
        temperature=0.5,
        huggingfacehub_api_token=HF_TOKEN,
        max_new_tokens=256,
    )
    return llm


hf_llm = load_llm(HUGGINGFACE_REPO_ID)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
from langchain.tools import Tool
from langchain_core.messages import HumanMessage, AIMessage
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent


#Database Loading

In [ ]:
DB_FAISS_PATH = "vectorstore/db_faiss"
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.load_local(DB_FAISS_PATH, embedding_model, allow_dangerous_deserialization=True)

#Prompt for RAG

In [ ]:

CUSTOM_PROMPT_TEMPLATE = """
Use the pieces of information provided in the context to answer the user's question.
If you don't know, just say so—don't fabricate.
Context: {context}
Question: {question}
Answer (concise, no chit-chat):
"""
prompt = PromptTemplate(template=CUSTOM_PROMPT_TEMPLATE, input_variables=["context", "question"])

#RAG_as_Tool

In [16]:
def build_rag_tool(vstore, llm):
    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vstore.as_retriever(search_kwargs={"k": 3}),
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt},
    )

    def _run(query: str) -> str:
        out = rag_chain.invoke({"query": query})
        answer = out["result"]
        sources = out.get("source_documents", [])

        source_texts = "\n\n".join(
            f"Source {i+1}:\n{doc.page_content}" for i, doc in enumerate(sources)
        )

        return f"{answer}\n\n---\nSources:\n{source_texts}"

    return Tool(
        name="KB-RAG",
        description="Answer questions using the internal knowledge base (PDFs).",
        func=_run,
    )


#Prompt_for_ReActive_Agent

In [35]:
REACT_PROMPT = """
You are a helpful assistant with access to these tools:
{tool_names}
Always greet professionally representing as an assistive Chatbot of ARB whenever user greets you.
When you receive a question, first think about whether you need to use a tool.
When you decide to call a tool mention the tool, use **exactly** this format (no variations):

Thought: do I need to use a tool? yes
Action: tool_name[tool input]

When the tool returns, continue like this:

Observation: tool output

Repeat Thought/Action/Observation as needed.  
When you are ready to answer the user, write:

Thought: Got the final answer <using the tool name here>.
Final answer: <your answer here, followed by “ Source: ” and the sources list including referred links, documents or any other resource>.

Begin!
"""


In [ ]:
#Add Content Safety as well. Look for AIGA Validation Framework
#Login History of Conversation

In [ ]:
search_tool = TavilySearchResults(max_results=2)
groq_llm = ChatGroq(model="llama-3.3-70b-versatile")
def get_response_from_ai_agent(
    query: str,
    *,
    allow_search=True,
    use_vector_db=True,
    recursion_limit=40,
):
    tools = []
    if allow_search:
        tools.append(search_tool)
    if use_vector_db:
        tools.append(rag_tool)
    #agent uses the Groq LLM
    agent = create_react_agent(
        model=groq_llm,
        tools=tools,
        messages_modifier=REACT_PROMPT
    )

    state = {"messages": [HumanMessage(content=query)]}
    response = agent.invoke(state, config={"recursion_limit": recursion_limit})

    ai_msgs = [m for m in response["messages"] if isinstance(m, AIMessage)]
    if not ai_msgs:
        raise RuntimeError(
            "Agent didn’t finish. Dump:\n" + "\n".join(str(m) for m in response["messages"])
        )

    return ai_msgs[-1].content


In [42]:
answer = get_response_from_ai_agent('''Hi!''')

print("\n *ARB_Chatbot's Response:\n", answer)

/var/folders/v5/7sp4jvq13bnfz2s5v60yf4p40000gn/T/ipykernel_50833/20080166.py:17: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.3.0. Use 'state_modifier' parameter instead.
  agent = create_react_agent(



 *ARB_Chatbot's Response:
 Hello! I'm an assistive Chatbot of ARB. It's nice to meet you. How can I assist you today?


In [41]:
answer = get_response_from_ai_agent('''Sure, please tell me when AD Reviews are done in the Life Cycle''')

print("\n *ARB_Chatbot's Response:\n", answer)

/var/folders/v5/7sp4jvq13bnfz2s5v60yf4p40000gn/T/ipykernel_50833/20080166.py:17: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.3.0. Use 'state_modifier' parameter instead.
  agent = create_react_agent(



 *ARB_Chatbot's Response:
 Thought: Got the final answer using KB-RAG.
Final answer: AD Reviews are typically conducted at various stages of the Life Cycle, including during the Initiation phase, Planning phase, Execution phase, and Closure phase. Source: KB-RAG
